In [ ]:

#Data preprocessing
#"label" is the important value that is to be extracted from the column DOMAIN_DATA

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
%matplotlib inline


log_dataset1 = pd.read_excel(STEAM02 - 12 julho.xlsx, sheet_name = 0, *, header=0, names=None, index_col=None, usecols=None, 
dtype=None, engine=None, converters=None, true_values=None, false_values=None, skiprows=None, 
nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, parse_dates=False, date_parser=_NoDefault.no_default, 
date_format=None, thousands=None, decimal='.', comment=None, skipfooter=0, storage_options=None, dtype_backend=_NoDefault.no_default)

log_dataset1.head()

In [ ]:
log_dataset1.shape

In [ ]:
log_dataset1.describe

In [ ]:
log_dataset1.info

In [ ]:
log_dataset2 = pd.read_excel(STEAM01 - 2 julho.xlsx, sheet_name = 0, *, header=0, names=None, index_col=None, usecols=None, 
dtype=None, engine=None, converters=None, true_values=None, false_values=None, skiprows=None, 
nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, parse_dates=False, date_parser=_NoDefault.no_default, 
date_format=None, thousands=None, decimal='.', comment=None, skipfooter=0, storage_options=None, dtype_backend=_NoDefault.no_default)

log_dataset2.head()

In [ ]:
log_dataset2.shape

In [ ]:
log_dataset2.describe

In [ ]:
log_dataset2.info

In [ ]:
#Drop unnecessary columns

log_dataset1 = log_dataset1.drop('COMPONENT', axis=1)
log_dataset1 = log_dataset1.drop('EVENT_DATE', axis=1)
log_dataset1 = log_dataset1.drop('CUSTOM_DATA', axis=1)

log_dataset2 = log_dataset2.drop('COMPONENT', axis=1)
log_dataset2 = log_dataset2.drop('EVENT_DATE', axis=1)
log_dataset2 = log_dataset2.drop('CUSTOM_DATA', axis=1)

In [ ]:
#Missing value check

log_dataset1.isnull().sum()

In [ ]:
#Missing value check

log_dataset2.isnull().sum()

In [ ]:
#Data Analysis

log_dataset1['DOMAIN_DATA'].unique()

In [ ]:
import json as json
from json import json.loads

domain_data = x.iloc[: , ['DOMAIN_DATA']]
string_dict = json.loads(domain_data)

length = len(string_dict)
for i in range(length+1):
    for key in string_dict:
        if key == "label":
            log_dataset1['LABEL'] = string_dict["label"].value

In [ ]:
#Now we have 4 columns as "EVENT_TIMESTAMP", "EVENT_TYPE", "DOMAIN_DATA", "LABEL"

log_dataset1.head()

In [ ]:
#Feature selection
#"Event timestamp" column will make up the x axis and
#"Label" will make up the y axis

x = log_dataset1.iloc[: , :-3].values
y = log_dataset1.iloc[: , -1].values

print(x)
print(y)

In [ ]:
#Detection Model - ISOLATION FOREST (for the fact that anamoly score is readily available in this ML package)
#Model building

from sklearn.ensemble import IsolationForest


df = log_dataset1

rs = np.random.RandomState(0)
clf = IsolationForest(max_samples=100,random_state=rs, contamination=.1) 
clf.fit(df)

if_scores = clf.decision_function(df)
if_anomalies = clf.predict(df)
if_anomalies = pd.Series(if_anomalies).replace([-1,1],[1,0])
if_anomalies = num[if_anomalies == 1]

In [ ]:
#Plotting the model for better visualization

plt.figure(figsize=(12,8))
plt.hist(if_scores)
plt.title(‘Histogram of Avg Anomaly Scores: Lower => More Anomalous’)

In [ ]:
#Plotting the model with x and y

cmap = np.array(['white','red'])
plt.scatter(x, y, c='white', s=20, edgecolor='k')
plt.scatter(if_anomalies.iloc[:,0],if_anomalies.iloc[:,1],c='red')
plt.xlabel('Event Timestamp')
plt.ylabel('Ousystem Event Labels')
plt.title('Outsystems Service Studio Anomalies')